# Wykład 1 - programowanie asynchroniczne

## Programowanie współbieżne

![alt text](../img/concurrency_example.webp "Concurrency")

*(obrazek wygenerowany przez DALL-E)*

**Programowanie współbieżne** (ang. concurrent programming) polega na wykonywaniu wielu zadań w tym samym czasie lub w sposób, który symuluje jednoczesne wykonywanie, aby zwiększyć efektywność działania programów. Celem jest optymalne wykorzystanie zasobów systemowych, takich jak procesory i pamięć, co pozwala przyspieszyć realizację złożonych zadań. W programowaniu współbieżnym istnieją różne podejścia, w tym programowanie synchroniczne, asynchroniczne i równoległe, które różnią się sposobem zarządzania wątkami, zadaniami i czasem ich wykonywania.

**Programowanie synchroniczne** (ang. synchronous programming) zakłada, że zadania są wykonywane jedno po drugim, w sposób sekwencyjny. Każde zadanie musi zakończyć się, zanim kolejne zostanie rozpoczęte. To podejście jest proste w implementacji, ale może prowadzić do marnowania zasobów systemowych, szczególnie w sytuacjach, gdy jedno zadanie czeka na zakończenie operacji wejścia/wyjścia (np. oczekiwanie na dane z bazy). Synchroniczność zapewnia przewidywalność, ale nie zawsze jest optymalna w aplikacjach wymagających szybkości i elastyczności.

**Programowanie asynchroniczne** (ang. asynchronous programming) polega na wykonywaniu zadań niezależnie od siebie, bez konieczności czekania na zakończenie jednego zadania, aby rozpocząć kolejne. W praktyce oznacza to, że program może kontynuować swoje działanie, nawet jeśli jedno z zadań wymaga np. długiego czasu oczekiwania na odpowiedź z serwera lub operację wejścia/wyjścia. W modelu asynchronicznym, zadania są uruchamiane równocześnie, a gdy jedno z nich zakończy się, wynik jest zwracany bez przerywania innych działań programu. Dzięki temu zwiększa się efektywność i responsywność aplikacji, co jest szczególnie ważne w przypadku aplikacji sieciowych, gdzie czas oczekiwania na dane może być nieprzewidywalny.

**Programowanie równoległe** (ang. parallel programming) zakłada wykonywanie wielu zadań jednocześnie na różnych rdzeniach procesora lub procesorów. W przeciwieństwie do asynchroniczności, która polega na nieblokującym wykonywaniu zadań, równoległość faktycznie dzieli pracę między procesory w celu maksymalnego przyspieszenia obliczeń. Jest to technika szczególnie użyteczna w zadaniach wymagających dużych mocy obliczeniowych, takich jak obróbka grafiki, analizy danych, uczenie maszynowe lub symulacje. Programowanie równoległe pozwala na osiągnięcie znacznie wyższej wydajności poprzez rozbicie zadań na mniejsze podzadania, które mogą być wykonywane równolegle, skracając tym samym czas potrzebny na zakończenie całego procesu.

### Co jest nie tak z równoległością?

#### Współdzielenie zasobów
W sytuacji gdy wiele zadań wykonuje się jednocześnie w różnych wątkach, istnieje ryzyko, że kilka z nich będzie próbowało uzyskać dostęp do tych samych danych w tym samym czasie. To może prowadzić do tzw. wyścigów wątków (ang. race conditions), gdzie wynik programu zależy od tego, które zadanie uzyska dostęp do zasobu jako pierwsze, co może prowadzić do nieprzewidywalnych błędów.

#### Synchronizacja wątków
Synchronizacja wątków polega na ustaleniu właściwej kolejności operacji. Nawet jeśli uda się skutecznie podzielić zadania na równoległe procesy, konieczne jest zapewnienie, że wyniki każdego z nich zostaną prawidłowo połączone na końcu. Błędy synchronizacji mogą prowadzić do blokad wzajemnych (zakleszczeń, ang. deadlocks), gdy minimum dwa wątki czekają na siebie w nieskończoność, co może całkowicie zatrzymać działanie programu.

#### Skomplikowane debugowanie
Błędy związane z równoczesnym wykonywaniem wielu zadań są trudniejsze do odtworzenia i zdiagnozowania, ponieważ ich występowanie może być zależne od subtelnych czynników, takich jak czas wykonania, dostępność zasobów czy układ procesora. Oznacza to, że testowanie aplikacji równoległych wymaga szczególnej uwagi i często skomplikowanych narzędzi do monitorowania przebiegu zadań.

#### GIL
Global Interpreter Lock (GIL) to mechanizm intepretera języka Python, który ogranicza wykonywanie wątków w interpreterze do jednego na raz, nawet na komputerach wielordzeniowych. Został wprowadzony na wczesnym etapie rozwoju Pythona, w latach '90, przez Guido van Rossuma, w głównej implementacji CPython, aby uprościć zarządzanie pamięcią i synchronizację wątków. Jego celem było zapobieganie równoczesnemu dostępowi do zasobów pamięci, co zwiększało bezpieczeństwo i upraszczało implementację. Od wersji 3.13 (planowanie wydanie: 07.10.2024) blokada GIL będzie opcjonalna i znacznie łatwiejsza do wyłączenia, niż we wcześniejszych wersjach interpretera.

## Programowanie asynchroniczne w języku Python

Problemów związanych z programowaniem równoległym można z łatwością uniknąć stosując podejście asynchroniczne. W znacznej większości przypadków programowania aplikacji sieciowych wystarczy jeden wątek obsługujący wiele zadań, które w większości czasu będą znajdowały się w stanie oczekiwania na odpowiedź z innej usługi sieciowej. Tego rodzaju podejście oszczędza nam znaczną ilość czasu, którą należałoby poświęcić na implementację rozwiązań unikających kolizji, czy zakleszczenia wątków. Warto mieć jednak na uwadze, że programowanie asynchroniczne wiąże się jednak również z dodatkowym wysiłkiem polegającym na obsłudze dodatkowych mechanizmów. Są one jednak nieporównywalnie łatwiejsze w zastosowaniu, nie wspominając już o deterinistycznym ich zachowaniu.

Język Python od swoich wczesnych wesji posiada wsparcie dla podejścia asynchronicznego. Obecnie najczęściej wykorzystywana jest biblioteka [asyncio](https://docs.python.org/3/library/asyncio.html), która od wersji 3.5 wchodzi w skład biblioteki standardowej języka Python. **Asyncio** umożliwia definiowanie funkcji asynchronicznych za pomocą słowa kluczowego `async def`, które mogą być wstrzymywane i wznawiane w różnych momentach za pomocą słowa `await`. Dzięki temu program może zająć się innymi zadaniami w trakcie oczekiwania na zakończenie długotrwałych operacji, poprawiając responsywność i wykorzystanie zasobów.

### Pierwszy program

In [ ]:
import asyncio


async def main() -> None:  # 1
    await asyncio.sleep(1)  # 2
    print("Hello world!")


if __name__ == "__main__":
    with asyncio.Runner() as runner:  # 3
        runner.run(main())  # 4

1. Pojawia się nowe słowo kluczowe `async def`, które oznacza definicję nowej `korutyny`, czyli funkcję z moliwością przepinania kontekstu wywołania.
2. Do "zawieszenia" korutyny wykorzystywana jest nieblokująca funkcja `sleep` pochodząca z biblioteki **asyncio**.
3. Nowa instancja klasy `Runner` w postaci managera kontekstu, która automatyzuje procesy niezbędne do uruchomienia asynchronicznego kodu.
4. Uruchomienie głównej korutyny.

Przedstawiony przykład można dodatkowo uprościć korzystając z funkcji `run`.

In [ ]:
import asyncio


async def main() -> None:
    await asyncio.sleep(1)
    print("Hello world!")


if __name__ == "__main__":
    asyncio.run(main())

Do wywołania głównej korutyny została wykorzystana funkcja `run` zamiast instancji klasy `Runner`, co zmniejsza objętość kodu źródłowego. Istnieją jednak istotne różnice między zastosowanymi podejściami.

1. Funkcja `run` automatycznie tworzy za każdym razem nową jednowątkową pętlę zdarzeń, a następnie zamyka ją gdy zadanie zostanie wykonane.
2. Klasa `Runner` jest rozwiązaniem wprowadzonym w wersji 3.11 języka Python umożliwiająca większą kontrolę nad pętlą zdarzeń.
3. Klasa `Runner` umożliwia uruchamianie wielu zadań w pętli zdarzeń (w różnych momentach), a także jej otwieranie i zamykanie.

![alt text](../img/event_loop.webp "Event loop")

### Pierwszy program w postaci mniej zautomatyzowanej

Stosowanie klasy `Runner` lub funkcji `run` nie są jedynymi rozwiązaniami uruchomienie kodu asynchronicznego. Operacje utworzenia, otwarcia i zamniecia pętli zdarzeń, a takż przekazywania i obsługi zadań mogą zostać wykonane manualnie.

In [ ]:
import asyncio


async def main() -> None:
    await asyncio.sleep(1)
    print("Hello world!")


if __name__ == "__main__":
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)  # 1
    
    task = loop.create_task(main())  # 2
    loop.run_until_complete(task)  # 3

    pending = asyncio.all_tasks(loop=loop)  # 4
    for pending_task in pending:
        pending_task.cancel()  # 5

    group = asyncio.gather(*pending, return_exceptions=True)  # 6
    loop.run_until_complete(group)  # 7

    loop.close()  # 8

- (1.) Powstaje nowa pętla zdarzeń, która następnie jest otwierana.
- (2.) Tworzone jest nowe zadanie oraz planowane jest jego uruchomienie w pętli zdarzeń.
- (3.) Blokowany jest wątek główny programu celem zapewnienia bezkolizyjnego działania pętli zdarzeń do czasu zakończenia działania wszystkich zaplanowanych zadań.
- (4., 5.) Zebranie wszystkich oczekujących zadań po odblokowaniu wątku głównego.
- (6.) Ponowne zebranie wszystkich zadań.
- (7.) Dokończenie wszystkich zebranych zadań.
- (8.) Zamknięcie pętli zdarzeń.

### Koprocedury

Korutyny (koprocedury) w Pythonie, realizowane za pomocą biblioteki **asyncio**, to funkcje asynchroniczne, które mogą wstrzymywać swoje działanie w trakcie wykonywania, umożliwiając innym zadaniom współbieżne działanie w tym samym czasie. Koprocedury definiuje się za pomocą słowa kluczowego `async def`, a ich wstrzymywanie i wznawianie odbywa się przez użycie `await`, które przekazuje kontrolę do pętli zdarzeń, bez blokowania całego programu. Dzięki temu, koprocedury pozwalają na efektywne zarządzanie operacjami wejścia/wyjścia, jak oczekiwanie na odpowiedzi z usług sieciowych, operacje na systemie plików czy bazach danych, bez zbędnego zużywania zasobów systemowych. 

In [ ]:
async def foo() -> int:
    return 10  # 1


async def main() -> None:
    result = await foo()  # 2
    print(result)  # 3

1. Nowa korutyna `foo` ma jedno zadanie - zwrócić wartość 10, w każdym przypadku. Nic nie stoi na przedzkodzie by umieścić w środku nieblokujący kod odwołujący się do innych usług sieciowych. Dzięki podejściu asynchronicznemu taka funkcja nie będzie blokowała wątku głównego aplikacji.
2. Pojawia się nowe słowo kluczowe `await`, tuż obok wywoływanej korutyny. Dzięki temu słowu kluczowemu możliwe jest przekazanie dalszego sterowania pętli zdarzeń, a sama operacja realizowana za pomocą korutyny delegowana jest do wykonywania w tle.
3. Po zakończeniu działania korutyny (zadania) foo zostanie zwrócony wynik, który następnie można wyświetlić jak każdy inny obiekt.

Język Python posiada pakiet `inspect`, który stanowi część biblioteki standardowej. Za jego pomocą można z łatwością sprawdzić czy dana funkcja lub metoda jest synchroniczna bądź asynchroniczna.

In [ ]:
import inspect

coro = foo()
inspect.iscoroutine(coro)

### Kończenie działania asynchronicznych programów

Programy asynchroniczne powinny mieć również wsparcie dla procesów ich kończenia. W znacznej większości przypadków programy komunikujące się współbieżnie z innymi usługami sieciowymi, czy systemem plików, powinny poprawnie zamykać wszelką otwartą komunikację. W przypadku stosowania biblioteki **asyncio** cały proces sprowadza się do przechwycenia momentu, w którym aplikacja kończy swoje działanie, a następnie uruchomieniu kodu odpowiedzialnego za zakończenie wszystkich zadań i zamknięcie pętli zdarzeń przed zakończeniem działania procesu.

In [ ]:
import asyncio


async def main() -> None:  # 1
    while True:
        print("it works")
        await asyncio.sleep(1)


if __name__ == "__main__":
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    task = loop.create_task(main())  # 2

    try:
        loop.run_until_complete(task)  # 3
    except KeyboardInterrupt:  # 4
        print("Closing the app")

        tasks = asyncio.all_tasks(loop=loop)  # 5
        for task_ in tasks:
            task_.cancel()
        
        group = asyncio.gather(*tasks, return_exceptions=True)
        loop.run_until_complete(group)
        loop.close()

1. Korutyna `main` uruchamia nieskończoną pętlę, co jest typowe dla działania usług sieciowych.
2. W przypadku przejęcia kontroli nad procesem kończenia owartych zadań i zamykania pętli zdarzeń, należy najpierw utworzyć te obiekty manualnie.
3. Wątek główny jest blokowany wewnątrz bloku `try` z uwagi na łapanie zdarzenia wyłączenia aplikacji w punkcie 4. za pomocą bloku `except`.
4. Złowienie wyjątku klasy `KeyboardInterrupt` odpowiada przechwyceniu akcji, gdy użytkownik wciśnie kombinację klawiszy `ctrl+c`. Działając w tym bloku można uruchomić kod, który wykona się jeszcze przed zakończeniem działania procesu.
5. Przed zakończeniem działania procesu należy wykonać standardowe kroki kończące aplikację asynchroniczną, czyli zgromadzenie i zgrupowanie wszystkich działających zadań, a następnie dokończenie ich i zamknięcie pętli zdarzeń.

## Zadania do wykonania na ćwiczeniach

Wszystkie zadania należy wykonać za pomocą biblioteki **asyncio**.

1. Utworzyć korutynę, która wstrzymuje działanie na 2 sekundy, a potem wyświetla komunikat o treści "Oczekiwanie zakończone".
2. Utwórz korutynę, która wyświetla wiadomość "Hello" po jednej sekundzie i "world" po dwóch sekundach.
3. Utworzyć dwie korutyny, a nstępnie je uruchomić współbienie. Obie korutyny będą miały takie samo działanie: oczekują zadaną ilość czasu, a następnie wyświetlają komunikat. Niech pierwsza z nich czeka trzy sekundy, a druga jedną sekundę.
4. Utworzyć aplikację, która co sekundę wyświetla kolejne liczby od 1 do 5. Należy pamiętać o zastosowaniu podejścia asynchronicznego!
5. Utworzyć aplikację, która będzie wykonywała się przez N sekund, co sekundę wyświetlając kolejną liczbę `ciągu Fibonacciego`.
6. Utworzyć aplikację, która będzie symulowała pobieranie danych z innych usług sieciowych. W tym celu należy utworzyć korutynę `fetch(delay: int)`, która po odczekaniu `delay` sekund zwróci dowolną wartość (symulacja pobierania danych z sieci). Następnie należy kilkukrotnie wywołać współbieżnie korutynę `fetch` z różnymi wartościami parametru `delay`. **Podpowiedź**: do współbieżnego wywołania wielu korutyn można wykorzystać funkcję `gather`, która zostanie umieszczona w dedykowanej korutynie. Wówczas nie ma potrzeby stosowania manualnego podejścia zarządzania zadaniami oraz pętlą zdarzeń.
7. Zadanie polega na symulowaniu kuchni, w której kilku kucharzy przygotowuje różne posiłki równocześnie. Każdy posiłek składa się z kilku etapów, np. krojenie warzyw, gotowanie makaronu, smażenie mięsa. Każdy etap trwa określony czas i jest realizowany asynchronicznie. Kucharze przygotowują trzy różne dania. Każde danie wymaga wykonania trzech kroków, które trwają różny czas (np. krojenie – 2 sekundy, gotowanie – 5 sekund, smażenie – 3 sekundy).
8. Zadanie polega na symulowaniu przetwarzania pięciu dużych plików, gdzie każdy plik musi przejść przez kilka etapów przetwarzania, takich jak wczytanie, analiza i zapis. Każdy z tych kroków trwa określony czas i musi być wykonany asynchronicznie. Każdy plik przechodzi przez trzy etapy: wczytanie (2 sekundy), analiza (4 sekundy), zapis (1 sekunda). Symuluj asynchroniczne przetwarzanie wszystkich plików naraz.
9. Zadanie polega na symulacji harmonogramu pracy w fabryce, gdzie różne maszyny wykonują swoje zadania w ustalonych przedziałach czasu. Maszyny muszą czekać na zakończenie poprzedniego cyklu zanim zaczną kolejny. Ustalić, aby każda maszyna działała w innym tempie, a wszystkie zadania były asynchroniczne. Każda maszyna ma swój cykl pracy, który powtarza się w określonym czasie (np. maszyna A – co 2 sekundy, maszyna B – co 3 sekundy, maszyna C – co 5 sekund). Należy zasymulować ich działanie przez 15 sekund.